In [12]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import json
import os
import shutil

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yaml
from PIL import Image
from tqdm import tqdm
import cv2
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import math
import ast

from ultralytics import YOLO

DATASET_ROOT = "/mnt/elice/dataset"  # 데이터 셋의 루트 디렉토리
LABELS_DIR = "labels"
IMAGE_DIR = "images"
TRAIN_DIR = "train"
TEST_DIR = "test"

### 학습 데이터 불러오기

In [ ]:
train_data = []

for image in tqdm(os.listdir(os.path.join(DATASET_ROOT, TRAIN_DIR, IMAGE_DIR))):
    image_id = image.split(".")[0]
    image_path = os.path.join(DATASET_ROOT, TRAIN_DIR, IMAGE_DIR, image)
    label_path = os.path.join(DATASET_ROOT, TRAIN_DIR, LABELS_DIR, image_id + ".txt")
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                class_id = int(line.split()[0])
                x = float(line.split()[1])
                y = float(line.split()[2])
                w = float(line.split()[3])
                h = float(line.split()[4])
                labels.append({"class_id": class_id, "x": x, "y": y, "w": w, "h": h})

    train_data.append({"id": image_id, "image_path": image_path, "label_path": label_path, "labels": labels})

df_train = pd.DataFrame(train_data)
df_train.head()

### 테스트 데이터 불러오기

In [ ]:
test_data = []

for image in tqdm(os.listdir(os.path.join(DATASET_ROOT, TEST_DIR, IMAGE_DIR))):
    image_id = image.split(".")[0]
    image_path = os.path.join(DATASET_ROOT, TEST_DIR, IMAGE_DIR, image)

    # 테스트 데이터는 라벨 파일이 없습니다.
    test_data.append({"id": image_id, "image_path": image_path, "label_path": "", "labels": []})

df_test = pd.DataFrame(test_data)
df_test.head()

## 클러스터링

In [1]:
def calculate_histogram(image_path, bins=256):
    # 이미지를 로드하고 HSV 컬러 공간으로 변환
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # 각 채널의 히스토그램 계산
    hist_hue = cv2.calcHist([hsv_image], [0], None, [bins], [0, 180])
    hist_saturation = cv2.calcHist([hsv_image], [1], None, [bins], [0, 256])
    hist_value = cv2.calcHist([hsv_image], [2], None, [bins], [0, 256])
    
    # 히스토그램 정규화 및 병합
    cv2.normalize(hist_hue, hist_hue)
    cv2.normalize(hist_saturation, hist_saturation)
    cv2.normalize(hist_value, hist_value)
    
    return np.concatenate([hist_hue.flatten(), hist_saturation.flatten(), hist_value.flatten()])

def bhattacharyya_distance(hist1, hist2):
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)

In [ ]:
train_id = df_train["id"].values
test_id = df_test["id"].values

df_concat = pd.concat([df_train, df_test], axis=0)
df_concat.reset_index(inplace = True)

attempt = df_concat.copy()
attempt["histograms"] = attempt["image_path"].apply(calculate_histogram)
histograms = np.array(attempt["histograms"].tolist())

num_images = len(histograms)
distance_matrix = np.zeros((num_images, num_images))

for i in range(num_images):
    for j in range(num_images):
        distance_matrix[i, j] = bhattacharyya_distance(histograms[i], histograms[j])
        
num_clusters = 3
kmeans = KMeans(n_clusters = num_clusters, random_state = 42, n_init = 10)
attempt["cluster"] = kmeans.fit_predict(diistance_matrix)

df_train = df_concat[df_concat["id"] == train_id]
df_train.reset_index(inplace = True)

df_test = df_concat[df_concat["id"] == test_id]
df_test.reset_index(inplace = True)

In [16]:
df_train0 = df_train.loc[df_train['cluster']==0] 
df_train1 = df_train.loc[df_train['cluster']==1] 
df_train2 = df_train.loc[df_train['cluster']==2] 

df_test0 = df_test.loc[df_test['cluster']==0]
df_test1 = df_test.loc[df_test['cluster']==1] 
df_test2 = df_test.loc[df_test['cluster']==2] 

## 0번 클러스터 학습

In [17]:
df_train0_set = df_train0.sample(frac=0.95, random_state=0)
df_valid0_set = df_train0.drop(df_train0_set.index)

df_train0_set.to_csv("train0_set.csv", index=False)
df_valid0_set.to_csv("valid0_set.csv", index=False)

new_train_path = os.path.abspath("train0")
new_valid_path = os.path.abspath("valid0")

os.makedirs(new_train_path, exist_ok=True)
os.makedirs(os.path.join(new_train_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_train_path, LABELS_DIR), exist_ok=True)

os.makedirs(new_valid_path, exist_ok=True)
os.makedirs(os.path.join(new_valid_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_valid_path, LABELS_DIR), exist_ok=True)

In [18]:
IMAGE_SIZE = 640

for i, row in tqdm(df_train0_set.iterrows(), total=len(df_train0_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_train_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_train_path}/{LABELS_DIR}/{row['id']}.txt")

for i, row in tqdm(df_valid0_set.iterrows(), total=len(df_valid0_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_valid_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_valid_path}/{LABELS_DIR}/{row['id']}.txt")

100%|██████████| 265/265 [00:05<00:00, 45.35it/s]


In [19]:
dataset = {
    "path": os.path.abspath("."),
    "train": "train0",
    "val": "valid0",
    "nc": 1,
    "names": ["coronal_hole"],
}

YAML_PATH = os.path.abspath("sun0.yaml")
RESULT_PATH = os.path.abspath("result0")
os.makedirs(RESULT_PATH, exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test0'), exist_ok=True)
with open(YAML_PATH, "w") as f:
    yaml.dump(dataset, f)

In [20]:
os.makedirs(os.path.join(RESULT_PATH, 'test0'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test0', 'exp'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test0', 'exp', 'labels'), exist_ok=True)

In [21]:
model0 = YOLO('yolov8n.yaml') 

In [ ]:
model0.train(data='sun0.yaml', epochs=1000, optimizer='AdamW', lr0=0.01, lrf=0.01, momentum=0.9, batch=128, 
                                 iou=0.5, patience=100, imgsz=640, augment=False, device=0,  pretrained=False)

New https://pypi.org/project/ultralytics/8.2.79 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=sun0.yaml, epochs=1500, time=None, patience=200, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train98, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning /home/elicer/train0/labels.cache... 5034 images, 167 backgrounds, 0 corrupt: 100%|██████████| 5034/5034 [00:00<?, ?it/s]
val: Scanning /home/elicer/valid0/labels.cache... 265 images, 6 backgrounds, 0 corrupt: 100%|██████████| 265/265 [00:00<?, ?it/s]


Plotting labels to runs/detect/train98/labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train98
Starting training for 1500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/1500      17.4G      3.297      3.082      3.313        122        640: 100%|██████████| 40/40 [00:17<00:00,  2.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.19it/s]

                   all        265        576    0.00494      0.115    0.00196   0.000451


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   492/1500      17.2G     0.7725     0.4622      1.112        166        640: 100%|██████████| 40/40 [00:13<00:00,  3.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.84it/s]

                   all        265        576      0.933      0.939       0.98       0.78



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   493/1500      17.2G     0.7687     0.4573      1.106        428        640:  75%|███████▌  | 30/40 [00:09<00:03,  3.07it/s]

In [27]:
model0 = YOLO('runs/detect/train98/weights/best.pt') 

In [28]:
new_test_path = os.path.abspath("test0")

os.makedirs(new_test_path, exist_ok=True)
os.makedirs(os.path.join(new_test_path, IMAGE_DIR), exist_ok=True)

In [30]:
IMAGE_SIZE = 640

for i, row in tqdm(df_test0.iterrows(), total=len(df_test0)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_test_path}/{IMAGE_DIR}/{row['id']}.jpg")


 63%|██████▎   | 1420/2269 [00:34<00:21, 39.06it/s]


100%|██████████| 2269/2269 [00:55<00:00, 40.77it/s]


In [31]:
RESULT_PATH = os.path.abspath("result0")
save_dir = os.path.join(RESULT_PATH, "test0", "exp")

results = model0.predict(
    source='test0/images/', 
    save=True, 
    save_txt=True, 
    project=save_dir,  
    name=""  
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2269 /home/elicer/test0/images/002138bcbd4343b6a6739c6f21f163d8.jpg: 640x640 2 coronal_holes, 5.8ms
image 2/2269 /home/elicer/test0/images/002c8ed220dc477493c43d35fefb71cf.jpg: 640x640 1 coronal_hole, 7.5ms
image 3/2269 /home/elicer/test0/images/0037cd24f811410ea5c7790db7d71f3d.jpg: 640x640 2 coronal_holes, 7.2ms
image 4/2269 /home/elicer/test0/images/0054b99ca25f4e9ea2b62f2418005679.jpg: 640x640 3 coronal_holes, 6.6ms
image 5/2269 /home/elicer

image 72/2269 /home/elicer/test0/images/081fa2df325d40dfa14c40ecee3e1f31.jpg: 640x640 1 coronal_hole, 5.7ms
image 73/2269 /home/elicer/test0/images/0837042aece94023bd1e5b3224ae7c32.jpg: 640x640 4 coronal_holes, 5.6ms
image 74/2269 /home/elicer/test0/images/0848be6c189b478cab47d31f16c4aee4.jpg: 640x640 3 coronal_holes, 5.6ms
image 75/2269 /home/elicer/test0/images/084b149ba3a74a71bd20b263ff4b356f.jpg: 640x640 1 coronal_hole, 6.0ms
image 76/2269 /home/elicer/test0/images/0890356f1b0d4ea9b8a8bda4c057ce88.jpg: 640x640 1 coronal_hole, 5.8ms
image 77/2269 /home/elicer/test0/images/08a0ec2685154dd5ad1f40a8ffa05bd2.jpg: 640x640 2 coronal_holes, 5.8ms
image 78/2269 /home/elicer/test0/images/08a89c7975904ee0a3ced537699931f5.jpg: 640x640 1 coronal_hole, 5.8ms
image 79/2269 /home/elicer/test0/images/08b78490d65646af93c8eca659c01a0a.jpg: 640x640 1 coronal_hole, 5.7ms
image 80/2269 /home/elicer/test0/images/08e153bc83f244bda1da40dc5da869a9.jpg: 640x640 2 coronal_holes, 5.8ms
image 81/2269 /home/elic

image 147/2269 /home/elicer/test0/images/10da42cbefa946e592998c36412cfb9d.jpg: 640x640 1 coronal_hole, 5.8ms
image 148/2269 /home/elicer/test0/images/10eafcd4992c49a989bbe6e45d79575e.jpg: 640x640 3 coronal_holes, 5.7ms
image 149/2269 /home/elicer/test0/images/10eb88dfbfc647f79d7ee6ce28be83c5.jpg: 640x640 2 coronal_holes, 5.7ms
image 150/2269 /home/elicer/test0/images/10fd8573c2a84d19a0e5c23b527fe5e5.jpg: 640x640 1 coronal_hole, 5.7ms
image 151/2269 /home/elicer/test0/images/1126342de2314ace884126aa5d2ef703.jpg: 640x640 2 coronal_holes, 5.7ms
image 152/2269 /home/elicer/test0/images/114c5e5986564f89a983c8777bd1b911.jpg: 640x640 2 coronal_holes, 5.7ms
image 153/2269 /home/elicer/test0/images/114d9e638b754c9fbd70aaea3951d307.jpg: 640x640 1 coronal_hole, 5.6ms
image 154/2269 /home/elicer/test0/images/115cd6035d1c45a6a57c4184130c5b0f.jpg: 640x640 2 coronal_holes, 5.7ms
image 155/2269 /home/elicer/test0/images/1196ed0ddb6a4f3ea846b9e0d4bb0677.jpg: 640x640 1 coronal_hole, 5.9ms
image 156/2269

image 222/2269 /home/elicer/test0/images/17a56d89c536472887cc4eb59c6312aa.jpg: 640x640 2 coronal_holes, 5.7ms
image 223/2269 /home/elicer/test0/images/17a8caf77a6f4be791e64e81604fcb79.jpg: 640x640 3 coronal_holes, 5.7ms
image 224/2269 /home/elicer/test0/images/17ae3cb272e04a5b80015eaf928aa10a.jpg: 640x640 2 coronal_holes, 5.7ms
image 225/2269 /home/elicer/test0/images/17ba45207bfc4532a9d873d1755895ef.jpg: 640x640 1 coronal_hole, 5.9ms
image 226/2269 /home/elicer/test0/images/17f66a3d8bbf47a380af604b5f1511ae.jpg: 640x640 3 coronal_holes, 5.7ms
image 227/2269 /home/elicer/test0/images/1818dc3319b04b8792a274e01a1c4569.jpg: 640x640 2 coronal_holes, 5.8ms
image 228/2269 /home/elicer/test0/images/18297d5fe5e447bf931e85ae31b32957.jpg: 640x640 2 coronal_holes, 5.7ms
image 229/2269 /home/elicer/test0/images/18303270f0284ad9a995e3e2216c7f58.jpg: 640x640 2 coronal_holes, 5.7ms
image 230/2269 /home/elicer/test0/images/1830b437f9fd4f49b5fc9de28f3ec92f.jpg: 640x640 2 coronal_holes, 5.7ms
image 231/2

image 297/2269 /home/elicer/test0/images/207a1486f17d4add95fd806b11fc627d.jpg: 640x640 3 coronal_holes, 5.7ms
image 298/2269 /home/elicer/test0/images/208fd20614b84059a820da1d36e2fdbe.jpg: 640x640 3 coronal_holes, 5.8ms
image 299/2269 /home/elicer/test0/images/20b0c10dca604615b9ed28a646cfa898.jpg: 640x640 2 coronal_holes, 5.8ms
image 300/2269 /home/elicer/test0/images/20e57d5f79994ba988d911410c312043.jpg: 640x640 2 coronal_holes, 5.7ms
image 301/2269 /home/elicer/test0/images/20f94504c4274cd2b6845890d51d3f2c.jpg: 640x640 2 coronal_holes, 5.8ms
image 302/2269 /home/elicer/test0/images/21346988b01c4283988bbd4effafe817.jpg: 640x640 2 coronal_holes, 5.7ms
image 303/2269 /home/elicer/test0/images/213d101e609441bab185069ab5119446.jpg: 640x640 1 coronal_hole, 5.7ms
image 304/2269 /home/elicer/test0/images/21887cac9bc84b529a30d9002cbc8616.jpg: 640x640 2 coronal_holes, 6.0ms
image 305/2269 /home/elicer/test0/images/21a344c7317e44daa6345d9de235c77b.jpg: 640x640 1 coronal_hole, 5.9ms
image 306/22

...

image 1935/2269 /home/elicer/test0/images/d93f3b76979c4fcaaef227d3bbfb1b54.jpg: 640x640 2 coronal_holes, 5.7ms
image 1936/2269 /home/elicer/test0/images/d95a849c9b9d4926a6aa24726c436870.jpg: 640x640 2 coronal_holes, 5.7ms
image 1937/2269 /home/elicer/test0/images/d9652970b8d34c3684b17aa1070ee405.jpg: 640x640 (no detections), 5.7ms
image 1938/2269 /home/elicer/test0/images/d970f94bdc044f0cb24fa76d2cd7934a.jpg: 640x640 3 coronal_holes, 5.7ms
image 1939/2269 /home/elicer/test0/images/d98124b9ea3b4b65815ccd0731feeb4b.jpg: 640x640 3 coronal_holes, 5.8ms
image 1940/2269 /home/elicer/test0/images/d98202c03d8f49478e9d630af1bb44bc.jpg: 640x640 4 coronal_holes, 5.8ms
image 1941/2269 /home/elicer/test0/images/d98d9429b24d41879e69b76f7da7f2cb.jpg: 640x640 2 coronal_holes, 5.8ms
image 1942/2269 /home/elicer/test0/images/d99b25eb16444b1dbd218272ae587b65.jpg: 640x640 2 coronal_holes, 5.8ms
image 1943/2269 /home/elicer/test0/images/d9c1de29f3e440e981f9df8934ba4039.jpg: 640x640 3 coronal_holes, 5.8ms
i

image 2009/2269 /home/elicer/test0/images/e116c096e8e647038dcc89bb59a6eb8f.jpg: 640x640 2 coronal_holes, 5.8ms
image 2010/2269 /home/elicer/test0/images/e14d6b926c9549e5bbe76ae6d513ecba.jpg: 640x640 2 coronal_holes, 5.8ms
image 2011/2269 /home/elicer/test0/images/e18828e2a0cf4f0c8b6b75b8eac5aef8.jpg: 640x640 1 coronal_hole, 5.9ms
image 2012/2269 /home/elicer/test0/images/e1bdd21d593c43bc84ee9d8c456b03dc.jpg: 640x640 1 coronal_hole, 6.0ms
image 2013/2269 /home/elicer/test0/images/e1e926464e6f4a68b866f4fda4061dce.jpg: 640x640 2 coronal_holes, 6.0ms
image 2014/2269 /home/elicer/test0/images/e222f24460ee4c3e8e5408a95de07c38.jpg: 640x640 2 coronal_holes, 5.9ms
image 2015/2269 /home/elicer/test0/images/e25e6843d31948a398ba2c4bb328b740.jpg: 640x640 2 coronal_holes, 5.8ms
image 2016/2269 /home/elicer/test0/images/e2abbf2f4a4f4bccb3a11d742b48bb7b.jpg: 640x640 6 coronal_holes, 5.8ms
image 2017/2269 /home/elicer/test0/images/e2b14009c42e4dd6afcb5daaa7405e4a.jpg: 640x640 3 coronal_holes, 5.9ms
ima

image 2083/2269 /home/elicer/test0/images/ea3217043614475facebfab88af8b323.jpg: 640x640 2 coronal_holes, 5.8ms
image 2084/2269 /home/elicer/test0/images/ea7f26b06d5d46f6bce79c20e0bd5484.jpg: 640x640 3 coronal_holes, 5.7ms
image 2085/2269 /home/elicer/test0/images/ea8237f526ba4a558f8389997e8b17f4.jpg: 640x640 3 coronal_holes, 5.8ms
image 2086/2269 /home/elicer/test0/images/eaa649e31d3240e58b6d174800186572.jpg: 640x640 3 coronal_holes, 5.8ms
image 2087/2269 /home/elicer/test0/images/eb42db4c44d74588a3610a232c3fb76d.jpg: 640x640 2 coronal_holes, 5.7ms
image 2088/2269 /home/elicer/test0/images/eb4cf20c6cf8468c97b2611f6af89c73.jpg: 640x640 2 coronal_holes, 5.9ms
image 2089/2269 /home/elicer/test0/images/eb5eb1f02e7744459b3a35d1ae9c5cd8.jpg: 640x640 4 coronal_holes, 5.9ms
image 2090/2269 /home/elicer/test0/images/eb82060da2214345adf84e9ce05d628d.jpg: 640x640 2 coronal_holes, 5.8ms
image 2091/2269 /home/elicer/test0/images/eb8701abc3d34ad5934439e68bcea8a1.jpg: 640x640 2 coronal_holes, 5.7ms
i

image 2157/2269 /home/elicer/test0/images/f368fb18b6d54bb6aee66aad2113faf1.jpg: 640x640 1 coronal_hole, 5.7ms
image 2158/2269 /home/elicer/test0/images/f3758acb60c6459788abc83ccc4b5833.jpg: 640x640 2 coronal_holes, 5.8ms
image 2159/2269 /home/elicer/test0/images/f3b91e9a280b45a4a470a9355fb895e8.jpg: 640x640 1 coronal_hole, 5.9ms
image 2160/2269 /home/elicer/test0/images/f3bc6c4c221b45aab3c64c48663e8d24.jpg: 640x640 4 coronal_holes, 5.8ms
image 2161/2269 /home/elicer/test0/images/f3d1abfdee974fc58b2691133de627aa.jpg: 640x640 1 coronal_hole, 5.8ms
image 2162/2269 /home/elicer/test0/images/f3ea8976348542dabbf2f8228967fd25.jpg: 640x640 2 coronal_holes, 5.7ms
image 2163/2269 /home/elicer/test0/images/f3fe4dd3e8a3492c9b27338b94c32519.jpg: 640x640 1 coronal_hole, 5.6ms
image 2164/2269 /home/elicer/test0/images/f440c8815e904f47a225800a585837d8.jpg: 640x640 4 coronal_holes, 5.6ms
image 2165/2269 /home/elicer/test0/images/f47e52f0be2b471ebedbabbc7a78eee4.jpg: 640x640 2 coronal_holes, 5.6ms
image

image 2231/2269 /home/elicer/test0/images/fbd46684bae84eada145b1ea25904be5.jpg: 640x640 3 coronal_holes, 5.8ms
image 2232/2269 /home/elicer/test0/images/fc0b66459cb74951ac1cd14246b9c824.jpg: 640x640 4 coronal_holes, 5.8ms
image 2233/2269 /home/elicer/test0/images/fc33129de70045b4bb6fc545c1aaa33d.jpg: 640x640 2 coronal_holes, 7.0ms
image 2234/2269 /home/elicer/test0/images/fc4b410661724025b578c9c6fec8509e.jpg: 640x640 2 coronal_holes, 5.7ms
image 2235/2269 /home/elicer/test0/images/fc5549b5bb124c83a30fb41a0d585211.jpg: 640x640 1 coronal_hole, 5.7ms
image 2236/2269 /home/elicer/test0/images/fc564064e45c42ed9c0d59ff03f1e43e.jpg: 640x640 4 coronal_holes, 5.7ms
image 2237/2269 /home/elicer/test0/images/fc81eb87a87a4a5ab221c36159259a08.jpg: 640x640 2 coronal_holes, 5.6ms
image 2238/2269 /home/elicer/test0/images/fca9c0805c1647f78a4426593a04198c.jpg: 640x640 3 coronal_holes, 5.8ms
image 2239/2269 /home/elicer/test0/images/fcac2d3d08ed4e2faf120698b71464f8.jpg: 640x640 (no detections), 5.8ms
im

In [32]:
for result in results:
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))
    print('\n{class num:counts} =', uniq_cnt_dict,'\n')
    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model0.names[int(c)])


{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 4} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 3} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 2} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 4} 

class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole
class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0 , class_name = coronal_hole

{class num:counts} = {0.0: 1} 

class num = 0

In [33]:
RESULT_PATH = os.path.abspath("result0")
for i, row in tqdm(df_test0.iterrows(), total=len(df_test0)):
    label_path = os.path.join(RESULT_PATH, "test0", "exp", "predict16", "labels", row["id"] + ".txt")

    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            class_id, c_x, c_y, w, h = map(float, line.split())
            labels.append({"class_id": int(class_id), "x": c_x, "y": c_y, "w": w, "h": h})

    df_test0.at[i, "labels"] = labels
df_test0


100%|██████████| 2269/2269 [00:00<00:00, 5958.11it/s]


,id,image_path,label_path,labels,histograms,cluster
4,4d02b3570c414445b34516800e5eef75,/mnt/elice/dataset/test/images/4d02b3570c41444...,NaN,"[{'class_id': 0, 'x': 0.514955, 'y': 0.114202,...",[6.64155930e-02 9.74660553e-03 6.68687820e-01 ...,1
5,b8c9fd8ff3894f178ee330c4ba285a95,/mnt/elice/dataset/test/images/b8c9fd8ff3894f1...,NaN,"[{'class_id': 0, 'x': 0.456221, 'y': 0.874741,...",[8.30249414e-02 1.73973218e-02 7.64732718e-01 ...,1
9,900171cc7641448bb3e546aa68ff16c3,/mnt/elice/dataset/test/images/900171cc7641448...,NaN,"[{'class_id': 0, 'x': 0.512821, 'y': 0.861852,...",[7.26769120e-02 1.13659929e-02 8.09445262e-01 ...,1
12,be395257cc8d4fab891d24672a6c1052,/mnt/elice/dataset/test/images/be395257cc8d4fa...,NaN,"[{'class_id': 0, 'x': 0.439076, 'y': 0.815674,...",[6.67001233e-02 7.07044033e-03 8.40975106e-01 ...,1
15,1712b54ce3b645058cec2120d60cd1ea,/mnt/elice/dataset/test/images/1712b54ce3b6450...,NaN,"[{'class_id': 0, 'x': 0.466244, 'y': 0.138028,...",[4.93633486e-02 8.74664076e-03 6.71375871e-01 ...,1
...,...,...,...,...,...,...
6351,ce7ac1c429f1425c8c7ff9b0dd5c3b21,/mnt/elice/dataset/test/images/ce7ac1c429f1425...,NaN,"[{'class_id': 0, 'x': 0.417885, 'y': 0.767249,...",[6.85279965e-02 2.01409925e-02 8.06880891e-01 ...,1
6353,5995ea87eed64fe8b8c546720c709d1b,/mnt/elice/dataset/test/images/5995ea87eed64fe...,NaN,"[{'class_id': 0, 'x': 0.723088, 'y': 0.712098,...",[7.06881046e-01 5.74407494e-03 9.90107749e-03 ...,1
6357,bca8844c96ef44a38145341e4a06cc91,/mnt/elice/dataset/test/images/bca8844c96ef44a...,NaN,"[{'class_id': 0, 'x': 0.717767, 'y': 0.436198,...",[4.64299284e-02 7.75896944e-03 4.85251099e-01 ...,1
6360,face0c0796cc479db13da2e870a3a875,/mnt/elice/dataset/test/images/face0c0796cc479...,NaN,"[{'class_id': 0, 'x': 0.628146, 'y': 0.533895,...",[5.27832322e-02 1.23279793e-02 6.40032768e-01 ...,1


## 1번 클러스터 예측

In [34]:
df_train1_set = df_train1.sample(frac=0.95, random_state=0)
df_valid1_set = df_train1.drop(df_train1_set.index)

df_train1_set.to_csv("train1_set.csv", index=False)
df_valid1_set.to_csv("valid1_set.csv", index=False)

new_train_path = os.path.abspath("train1")
new_valid_path = os.path.abspath("valid1")

os.makedirs(new_train_path, exist_ok=True)
os.makedirs(os.path.join(new_train_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_train_path, LABELS_DIR), exist_ok=True)

os.makedirs(new_valid_path, exist_ok=True)
os.makedirs(os.path.join(new_valid_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_valid_path, LABELS_DIR), exist_ok=True)

In [35]:
IMAGE_SIZE = 640

for i, row in tqdm(df_train1_set.iterrows(), total=len(df_train1_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_train_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_train_path}/{LABELS_DIR}/{row['id']}.txt")

for i, row in tqdm(df_valid1_set.iterrows(), total=len(df_valid1_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_valid_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_valid_path}/{LABELS_DIR}/{row['id']}.txt")


 32%|███▏      | 1297/4036 [00:34<01:08, 39.81it/s]


 64%|██████▍   | 2582/4036 [01:08<00:36, 39.99it/s]


 95%|█████████▍| 3829/4036 [01:41<00:05, 39.55it/s]


100%|██████████| 4036/4036 [01:46<00:00, 37.83it/s]

100%|██████████| 212/212 [00:05<00:00, 40.00it/s]


In [36]:
dataset = {
    "path": os.path.abspath("."),
    "train": "train1",
    "val": "valid1",
    "nc": 3,
    "names": ["coronal_hole", "sunspot", "prominence"],
}

YAML_PATH = os.path.abspath("sun1.yaml")
RESULT_PATH = os.path.abspath("result1")
os.makedirs(RESULT_PATH, exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test1'), exist_ok=True)
with open(YAML_PATH, "w") as f:
    yaml.dump(dataset, f)

In [37]:
os.makedirs(os.path.join(RESULT_PATH, 'test1'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test1', 'exp'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test1', 'exp', 'labels'), exist_ok=True)

In [38]:
model1 = YOLO('yolov8n.yaml') 

In [39]:
model1.train(data='sun1.yaml', epochs=1000, optimizer='AdamW', lr0=0.01, lrf=0.01, momentum=0.9, batch=128, 
                                 iou=0.5, patience=100, imgsz=640, augment=False, device=0,  pretrained=False)

New https://pypi.org/project/ultralytics/8.2.79 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=sun1.yaml, epochs=1200, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train101, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save


train: Scanning /home/elicer/train1/labels...:   0%|          | 0/4036 [00:00<?, ?it/s]
train: Scanning /home/elicer/train1/labels... 190 images, 5 backgrounds, 0 corrupt:   5%|▍         | 190/4036 [00:00<00:02, 1841.21it/s]
train: Scanning /home/elicer/train1/labels... 398 images, 11 backgrounds, 0 corrupt:  10%|▉         | 398/4036 [00:00<00:01, 1959.59it/s]
train: Scanning /home/elicer/train1/labels... 595 images, 17 backgrounds, 0 corrupt:  15%|█▍        | 595/4036 [00:00<00:01, 1811.20it/s]
train: Scanning /home/elicer/train1/labels... 778 images, 25 backgrounds, 0 corrupt:  19%|█▉        | 778/4036 [00:00<00:01, 1785.42it/s]
train: Scanning /home/elicer/train1/labels... 958 images, 37 backgrounds, 0 corrupt:  24%|██▎       | 958/4036 [00:00<00:01, 1778.93it/s]
train: Scanning /home/elicer/train1/labels... 1137 images, 45 backgrounds, 0 corrupt:  28%|██▊       | 1137/4036 [00:00<00:01, 1757.71it/s]
train: Scanning /home/elicer/train1/labels... 1313 images, 51 backgrounds, 0 corru

train: New cache created: /home/elicer/train1/labels.cache



val: Scanning /home/elicer/valid1/labels...:   0%|          | 0/212 [00:00<?, ?it/s]
val: Scanning /home/elicer/valid1/labels... 212 images, 3 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<00:00, 1662.81it/s]

val: New cache created: /home/elicer/valid1/labels.cache


...

                   all        212        819      0.931      0.933      0.971       0.66
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 643, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



743 epochs completed in 2.430 hours.
Optimizer stripped from runs/detect/train101/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train101/weights/best.pt, 6.3MB

Validating runs/detect/train101/weights/best.pt...
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8n summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


                   all        212        819      0.917      0.949      0.972      0.662
               sunspot        209        819      0.917      0.949      0.972      0.662
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train101


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f9427732a70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [40]:
model1 = YOLO('runs/detect/train101/weights/best.pt') 

In [41]:
new_test_path = os.path.abspath("test1")

os.makedirs(new_test_path, exist_ok=True)
os.makedirs(os.path.join(new_test_path, IMAGE_DIR), exist_ok=True)

In [42]:
IMAGE_SIZE = 640

for i, row in tqdm(df_test1.iterrows(), total=len(df_test1)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_test_path}/{IMAGE_DIR}/{row['id']}.jpg")

100%|██████████| 1819/1819 [00:52<00:00, 34.70it/s]


In [43]:
RESULT_PATH = os.path.abspath("result1")
save_dir = os.path.join(RESULT_PATH, "test1", "exp")

results = model1.predict(
    source='test1/images/', 
    save=True, 
    save_txt=True, 
    project=save_dir,  
    name=""  
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1819 /home/elicer/test1/images/001f5cc5f47541518b712d6b0a6d18cc.jpg: 640x640 5 sunspots, 5.5ms
image 2/1819 /home/elicer/test1/images/002c625b772e431ba42baa5820d119bc.jpg: 640x640 6 sunspots, 5.4ms
image 3/1819 /home/elicer/test1/images/002e50622f1c46d99ad24c3f939b8067.jpg: 640x640 1 sunspot, 5.5ms
image 4/1819 /home/elicer/test1/images/00336d0e812746608acda25d943b38d0.jpg: 640x640 2 sunspots, 5.4ms
image 5/1819 /home/elicer/test1/images/004973

image 75/1819 /home/elicer/test1/images/0a18a7c3618d4e3e9138fe4d1645eaba.jpg: 640x640 2 sunspots, 5.4ms
image 76/1819 /home/elicer/test1/images/0a1f945d15974417bf5822b1b30862c6.jpg: 640x640 5 sunspots, 5.4ms
image 77/1819 /home/elicer/test1/images/0a34305800fa4c449f9c6b5a6b5620db.jpg: 640x640 (no detections), 5.4ms
image 78/1819 /home/elicer/test1/images/0a54ce46977f4b698b8b0dee9be024e9.jpg: 640x640 4 sunspots, 5.3ms
image 79/1819 /home/elicer/test1/images/0a6d3829818d4fc6b51066f0d6c981d2.jpg: 640x640 4 sunspots, 5.3ms
image 80/1819 /home/elicer/test1/images/0a70003161ad403296f125a338c19a66.jpg: 640x640 2 sunspots, 5.5ms
image 81/1819 /home/elicer/test1/images/0b4d820f9bc14057b232a9f98a88c715.jpg: 640x640 2 sunspots, 5.4ms
image 82/1819 /home/elicer/test1/images/0b929b77b1bd4652954e79b556ee773c.jpg: 640x640 5 sunspots, 5.4ms
image 83/1819 /home/elicer/test1/images/0bf52e74133f4e79870e2349cb1f734e.jpg: 640x640 4 sunspots, 5.4ms
image 84/1819 /home/elicer/test1/images/0c664fb963df408c8c0

image 154/1819 /home/elicer/test1/images/18f4ce580d014d76b22cdf0d568de78a.jpg: 640x640 1 sunspot, 6.0ms
image 155/1819 /home/elicer/test1/images/19010e3490ae4734aa34596a00268326.jpg: 640x640 7 sunspots, 6.1ms
image 156/1819 /home/elicer/test1/images/192492957c5d44e98675644c30386e06.jpg: 640x640 5 sunspots, 6.0ms
image 157/1819 /home/elicer/test1/images/19507a0780884473a6244d946cfd1fff.jpg: 640x640 4 sunspots, 6.0ms
image 158/1819 /home/elicer/test1/images/1973bfc1c0cd498981eb5b852f27398a.jpg: 640x640 2 sunspots, 6.1ms
image 159/1819 /home/elicer/test1/images/199ced8f48214f37b05b9b7e26d3505a.jpg: 640x640 11 sunspots, 6.0ms
image 160/1819 /home/elicer/test1/images/19c59dce168b4ca1b177295721968717.jpg: 640x640 5 sunspots, 6.0ms
image 161/1819 /home/elicer/test1/images/1a349969ece443f7af8c2e2a436130e7.jpg: 640x640 2 sunspots, 6.3ms
image 162/1819 /home/elicer/test1/images/1a41bfb1b64340d88326f7532bc4f8e8.jpg: 640x640 4 sunspots, 6.0ms
image 163/1819 /home/elicer/test1/images/1a4dffbeffa149

image 232/1819 /home/elicer/test1/images/24d62d51be4f48ffb7a4f2ab87faa059.jpg: 640x640 7 sunspots, 6.1ms
image 233/1819 /home/elicer/test1/images/24d686b0bcd242a082b209631b190a8c.jpg: 640x640 2 sunspots, 6.0ms
image 234/1819 /home/elicer/test1/images/24d804aff36f47ba9ae18976e79ef271.jpg: 640x640 10 sunspots, 6.1ms
image 235/1819 /home/elicer/test1/images/250106fbb60141168f3d39b26e737280.jpg: 640x640 3 sunspots, 6.0ms
image 236/1819 /home/elicer/test1/images/2505b9353e2e48c69188f2c23ffaff43.jpg: 640x640 5 sunspots, 6.1ms
image 237/1819 /home/elicer/test1/images/251c416adea849ce838b5f7ba67b3236.jpg: 640x640 2 sunspots, 6.1ms
image 238/1819 /home/elicer/test1/images/255ca28e23db4a2abf0570ba864098e9.jpg: 640x640 5 sunspots, 6.1ms
image 239/1819 /home/elicer/test1/images/255ca733c3bb41e69ae31203809a645b.jpg: 640x640 6 sunspots, 6.0ms
image 240/1819 /home/elicer/test1/images/25855ac9fe024b95a2b82c696ec590d8.jpg: 640x640 10 sunspots, 6.1ms
image 241/1819 /home/elicer/test1/images/25f49cfbfb4d

image 310/1819 /home/elicer/test1/images/30159eb31a2c4aa9aff353400f4c9399.jpg: 640x640 6 sunspots, 5.6ms
image 311/1819 /home/elicer/test1/images/303a08a6a54e4b83b7410023c644c93f.jpg: 640x640 2 sunspots, 5.6ms
image 312/1819 /home/elicer/test1/images/303ceb2d6c5246db990ae8659d3162f5.jpg: 640x640 3 sunspots, 5.6ms
image 313/1819 /home/elicer/test1/images/30aa6912de0e4b2a864c7f9ee4d1e460.jpg: 640x640 3 sunspots, 5.9ms
image 314/1819 /home/elicer/test1/images/30b249c852e84befa20ef921b49aef11.jpg: 640x640 3 sunspots, 5.8ms
image 315/1819 /home/elicer/test1/images/30bb8668b1074aa5bfefbf035e1926d8.jpg: 640x640 1 sunspot, 5.7ms
image 316/1819 /home/elicer/test1/images/30c04aeb1b774004b0b0dc9b71f2ac54.jpg: 640x640 1 sunspot, 5.6ms
image 317/1819 /home/elicer/test1/images/311a3182ab414d118146c370a7af5cfa.jpg: 640x640 8 sunspots, 5.6ms
image 318/1819 /home/elicer/test1/images/312a25b9082241cd8eba1d4fe13f5686.jpg: 640x640 2 sunspots, 5.7ms
image 319/1819 /home/elicer/test1/images/3134ef8ad9fd461a

...

image 1482/1819 /home/elicer/test1/images/cc2506d17cc14ff8b7261ecba2bb0cbc.jpg: 640x640 4 sunspots, 5.9ms
image 1483/1819 /home/elicer/test1/images/cc51fd66ba8c4b7ebb32bc0c4727964c.jpg: 640x640 4 sunspots, 5.5ms
image 1484/1819 /home/elicer/test1/images/cc669b4b8d674481944b621f1c1d23df.jpg: 640x640 6 sunspots, 5.7ms
image 1485/1819 /home/elicer/test1/images/cc7d56ac13884f0fb046a488a072a140.jpg: 640x640 4 sunspots, 5.6ms
image 1486/1819 /home/elicer/test1/images/cc9273fb26e441de88a18dc736d3d66a.jpg: 640x640 4 sunspots, 5.5ms
image 1487/1819 /home/elicer/test1/images/cc9ae36028c74c97a4e5776a4f2b306a.jpg: 640x640 5 sunspots, 5.4ms
image 1488/1819 /home/elicer/test1/images/cd218a55460e4c81a78cffe7dca3611a.jpg: 640x640 1 sunspot, 5.4ms
image 1489/1819 /home/elicer/test1/images/cd2939a443a94b8c86caa7773cd930f6.jpg: 640x640 5 sunspots, 5.4ms
image 1490/1819 /home/elicer/test1/images/cd66fda1d6d147dda245611f8e8d1fcf.jpg: 640x640 1 sunspot, 5.4ms
image 1491/1819 /home/elicer/test1/images/cd6c0c

image 1560/1819 /home/elicer/test1/images/d91da60c4eef4a11bfab1a37e2a440c6.jpg: 640x640 4 sunspots, 5.4ms
image 1561/1819 /home/elicer/test1/images/d9635ff20bb84dbb96264f6eefc21e08.jpg: 640x640 4 sunspots, 5.4ms
image 1562/1819 /home/elicer/test1/images/d9a4730ee5e34711a4b1936c33affbd9.jpg: 640x640 3 sunspots, 5.4ms
image 1563/1819 /home/elicer/test1/images/da04dc7090b2447ea40b64dcca724985.jpg: 640x640 5 sunspots, 5.4ms
image 1564/1819 /home/elicer/test1/images/da3f3cf52f89474fb855d178731e9330.jpg: 640x640 6 sunspots, 5.4ms
image 1565/1819 /home/elicer/test1/images/daa36804ced1402b8be7c4c9d3e39ca6.jpg: 640x640 2 sunspots, 5.4ms
image 1566/1819 /home/elicer/test1/images/daab01e1ba2b4e55b130f818d162edad.jpg: 640x640 7 sunspots, 5.4ms
image 1567/1819 /home/elicer/test1/images/dac5af956c204f2b8eaa4e0f0ac8de43.jpg: 640x640 8 sunspots, 5.5ms
image 1568/1819 /home/elicer/test1/images/daf5b9da37674c15a1ef8094ec555ea7.jpg: 640x640 5 sunspots, 5.5ms
image 1569/1819 /home/elicer/test1/images/daf9

image 1638/1819 /home/elicer/test1/images/e3268762d86c427a9e3856f04ce8ca55.jpg: 640x640 6 sunspots, 5.4ms
image 1639/1819 /home/elicer/test1/images/e326e7fa9c0b4acea3b42c45e6632212.jpg: 640x640 1 sunspot, 6.0ms
image 1640/1819 /home/elicer/test1/images/e33260998c154cb3937f3ca7d803a212.jpg: 640x640 4 sunspots, 5.5ms
image 1641/1819 /home/elicer/test1/images/e386c1167a764afbabf3dc209b808abb.jpg: 640x640 2 sunspots, 5.4ms
image 1642/1819 /home/elicer/test1/images/e38a7a4fb91d4659954de4d59147c15e.jpg: 640x640 2 sunspots, 5.5ms
image 1643/1819 /home/elicer/test1/images/e3b4ad83414e49d9a8019e2f5aff4059.jpg: 640x640 3 sunspots, 5.4ms
image 1644/1819 /home/elicer/test1/images/e3d0c3c5361c42b79ad188261cf0cb94.jpg: 640x640 9 sunspots, 5.5ms
image 1645/1819 /home/elicer/test1/images/e3e9e3473f5d42a1bf3a3f9ccd860dee.jpg: 640x640 5 sunspots, 5.5ms
image 1646/1819 /home/elicer/test1/images/e49f797a15b74ab18d68976b47c3b5ae.jpg: 640x640 3 sunspots, 5.4ms
image 1647/1819 /home/elicer/test1/images/e4e43

image 1716/1819 /home/elicer/test1/images/eef108135ca24eb8b3995450912bc738.jpg: 640x640 2 sunspots, 5.3ms
image 1717/1819 /home/elicer/test1/images/ef6afad926f643c7a4bd31c55e37af03.jpg: 640x640 4 sunspots, 5.4ms
image 1718/1819 /home/elicer/test1/images/ef72fb96462f4535b86ae9abbc5ec3f7.jpg: 640x640 2 sunspots, 5.4ms
image 1719/1819 /home/elicer/test1/images/efaf5ecd4ec148abad30b3647ca67279.jpg: 640x640 (no detections), 5.4ms
image 1720/1819 /home/elicer/test1/images/efd2fc50c23a472fadedf6cc2415f859.jpg: 640x640 4 sunspots, 5.3ms
image 1721/1819 /home/elicer/test1/images/effbfca1a4fc446096322bd4f28a5772.jpg: 640x640 4 sunspots, 5.4ms
image 1722/1819 /home/elicer/test1/images/f0088f18d2af4b31bdd85681ce1afed0.jpg: 640x640 3 sunspots, 5.4ms
image 1723/1819 /home/elicer/test1/images/f00a0b0af98543b29d386359c4d08018.jpg: 640x640 3 sunspots, 5.4ms
image 1724/1819 /home/elicer/test1/images/f01380d279414a3e9845bbf12693620f.jpg: 640x640 2 sunspots, 5.4ms
image 1725/1819 /home/elicer/test1/images

image 1794/1819 /home/elicer/test1/images/fcfbc85f0fb844d680f508cfef8883d7.jpg: 640x640 1 sunspot, 5.5ms
image 1795/1819 /home/elicer/test1/images/fd651550fba54e64a10a4b8d2df3ae57.jpg: 640x640 7 sunspots, 5.5ms
image 1796/1819 /home/elicer/test1/images/fd6843e13d864c8194b160631575ab7b.jpg: 640x640 4 sunspots, 5.5ms
image 1797/1819 /home/elicer/test1/images/fd9431500fcd4c6c8b967fed8b75deef.jpg: 640x640 5 sunspots, 5.5ms
image 1798/1819 /home/elicer/test1/images/fda3d2d3656642d59bc40523abbd9379.jpg: 640x640 1 sunspot, 6.5ms
image 1799/1819 /home/elicer/test1/images/fdd1d1fedd3d4d5ea3119b7232d060c9.jpg: 640x640 3 sunspots, 5.7ms
image 1800/1819 /home/elicer/test1/images/fdf16266e88348c88100e9d8fd932ba6.jpg: 640x640 4 sunspots, 5.5ms
image 1801/1819 /home/elicer/test1/images/fe20508c4687433384ed35374a225e50.jpg: 640x640 2 sunspots, 5.4ms
image 1802/1819 /home/elicer/test1/images/fe27ee46e77a4a2f8d93cce6d8dc93ae.jpg: 640x640 2 sunspots, 5.4ms
image 1803/1819 /home/elicer/test1/images/fe5ee3

In [44]:
for result in results:
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))
    print('\n{class num:counts} =', uniq_cnt_dict,'\n')
    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model1.names[int(c)])


{class num:counts} = {1.0: 5} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 6} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 1} 

class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 2} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 7} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 7} 

class num = 1 , class_nam

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 3} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 5} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 3} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 3} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 2} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 2} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 3} 

class num = 1 , class_name = 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 6} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 1} 

class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 5} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 2} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 1} 

class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 3} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 2} 

class num = 1 , class_name = 

class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 6} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 4} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 4} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 7} 

class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot
class num = 1 , class_name = sunspot

{class num:counts} = {1.0: 4} 

class num = 1 , class

In [45]:
RESULT_PATH = os.path.abspath("result1")
for i, row in tqdm(df_test1.iterrows(), total=len(df_test1)):
    label_path = os.path.join(RESULT_PATH, "test1", "exp", "predict15", "labels", row["id"] + ".txt")

    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            class_id, c_x, c_y, w, h = map(float, line.split())
            labels.append({"class_id": int(class_id), "x": c_x, "y": c_y, "w": w, "h": h})

    df_test1.at[i, "labels"] = labels
df_test1

100%|██████████| 1819/1819 [00:00<00:00, 6156.25it/s]


,id,image_path,label_path,labels,histograms,cluster
0,8abb1e02042142928d5280eb2137d538,/mnt/elice/dataset/test/images/8abb1e020421429...,NaN,"[{'class_id': 1, 'x': 0.777907, 'y': 0.594818,...",[7.28213787e-01 3.71094065e-05 3.76395386e-04 ...,2
3,29da3a3a93ec4c71b5415cfaac520831,/mnt/elice/dataset/test/images/29da3a3a93ec4c7...,NaN,"[{'class_id': 1, 'x': 0.347453, 'y': 0.48405, ...",[7.52214730e-01 1.07547239e-04 2.71312339e-04 ...,2
7,42d825d304e84a3a8b98def23abc5136,/mnt/elice/dataset/test/images/42d825d304e84a3...,NaN,"[{'class_id': 1, 'x': 0.421722, 'y': 0.270305,...",[6.38356209e-01 1.06070373e-04 2.39275032e-04 ...,2
8,8e2e7f45efb34454be1d4eab318ec73a,/mnt/elice/dataset/test/images/8e2e7f45efb3445...,NaN,"[{'class_id': 1, 'x': 0.45192, 'y': 0.606232, ...",[7.78014004e-01 3.56043711e-05 3.96734424e-04 ...,2
10,b4a2bc556aea4ee1b634cbe9ef06049a,/mnt/elice/dataset/test/images/b4a2bc556aea4ee...,NaN,"[{'class_id': 1, 'x': 0.59399, 'y': 0.68618, '...",[7.27917910e-01 1.14770934e-04 1.98458074e-04 ...,2
...,...,...,...,...,...,...
6356,f8ccf518915b4e289d9be23219c879b4,/mnt/elice/dataset/test/images/f8ccf518915b4e2...,NaN,"[{'class_id': 1, 'x': 0.765992, 'y': 0.379832,...",[6.59319043e-01 9.95620067e-05 1.75315698e-04 ...,2
6359,f62a733bf7804822b40d6812c5b7b2b3,/mnt/elice/dataset/test/images/f62a733bf780482...,NaN,"[{'class_id': 1, 'x': 0.05924, 'y': 0.393671, ...",[6.63427532e-01 1.21532372e-04 1.54677560e-04 ...,2
6366,2715986383dc466ca338bd7167ce1fd6,/mnt/elice/dataset/test/images/2715986383dc466...,NaN,"[{'class_id': 1, 'x': 0.224852, 'y': 0.699215,...",[7.57285953e-01 4.15848408e-05 3.37876845e-04 ...,2
6367,e94fad68d94348168d8cf3482e49f7b3,/mnt/elice/dataset/test/images/e94fad68d943481...,NaN,"[{'class_id': 1, 'x': 0.417555, 'y': 0.63731, ...",[7.11751282e-01 7.23343546e-05 3.45597451e-04 ...,2


## 2번 클러스터 예측

In [46]:
df_train2_set = df_train2.sample(frac=0.95, random_state=0)
df_valid2_set = df_train2.drop(df_train2_set.index)

df_train2_set.to_csv("train2_set.csv", index=False)
df_valid2_set.to_csv("valid2_set.csv", index=False)

new_train_path = os.path.abspath("train2")
new_valid_path = os.path.abspath("valid2")

os.makedirs(new_train_path, exist_ok=True)
os.makedirs(os.path.join(new_train_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_train_path, LABELS_DIR), exist_ok=True)

os.makedirs(new_valid_path, exist_ok=True)
os.makedirs(os.path.join(new_valid_path, IMAGE_DIR), exist_ok=True)
os.makedirs(os.path.join(new_valid_path, LABELS_DIR), exist_ok=True)

In [47]:
IMAGE_SIZE = 640

for i, row in tqdm(df_train2_set.iterrows(), total=len(df_train2_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_train_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_train_path}/{LABELS_DIR}/{row['id']}.txt")

for i, row in tqdm(df_valid2_set.iterrows(), total=len(df_valid2_set)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_valid_path}/{IMAGE_DIR}/{row['id']}.jpg")
    shutil.copy(row["label_path"], f"{new_valid_path}/{LABELS_DIR}/{row['id']}.txt")

100%|██████████| 266/266 [00:06<00:00, 44.00it/s]


In [48]:
dataset = {
    "path": os.path.abspath("."),
    "train": "train2",
    "val": "valid2",
    "nc": 3,
    "names": ["coronal_hole", "sunspot", "prominence"],
}

YAML_PATH = os.path.abspath("sun2.yaml")
RESULT_PATH = os.path.abspath("result2")
os.makedirs(RESULT_PATH, exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test2'), exist_ok=True)
with open(YAML_PATH, "w") as f:
    yaml.dump(dataset, f)

In [49]:
os.makedirs(os.path.join(RESULT_PATH, 'test2'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test2', 'exp'), exist_ok=True)
os.makedirs(os.path.join(RESULT_PATH, 'test2', 'exp', 'labels'), exist_ok=True)

In [50]:
model2 = YOLO('yolov8n.yaml') 

In [53]:
model2.train(data='sun2.yaml', epochs=1000, optimizer='AdamW', lr0=0.01, lrf=0.01, momentum=0.9, batch=128, 
                                 iou=0.5, patience=100, imgsz=640, augment=True, device=0,  pretrained=False)

New https://pypi.org/project/ultralytics/8.2.80 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=sun2.yaml, epochs=1000, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train104, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning /home/elicer/train2/labels.cache... 5054 images, 1036 backgrounds, 0 corrupt: 100%|██████████| 5054/5054 [00:00<?, ?it/s]
val: Scanning /home/elicer/valid2/labels.cache... 835 images, 197 backgrounds, 0 corrupt: 100%|██████████| 835/835 [00:00<?, ?it/s]


Plotting labels to runs/detect/train104/labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train104
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/1000      53.6G      3.591      4.115      3.169        129        640: 100%|██████████| 40/40 [00:14<00:00,  2.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.76it/s]

                   all        835       1245          0          0          0          0


...

                   all        835       1245      0.747      0.718      0.788      0.477
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 345, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



445 epochs completed in 1.945 hours.
Optimizer stripped from runs/detect/train104/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train104/weights/best.pt, 6.3MB

Validating runs/detect/train104/weights/best.pt...
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8n summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


                   all        835       1245      0.759      0.741       0.82      0.499
            prominence        638       1245      0.759      0.741       0.82      0.499
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train104


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f930e7fc190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [54]:
model2 = YOLO('runs/detect/train104/weights/best.pt') 

In [55]:
new_test_path = os.path.abspath("test2")

os.makedirs(new_test_path, exist_ok=True)
os.makedirs(os.path.join(new_test_path, IMAGE_DIR), exist_ok=True)

In [56]:
IMAGE_SIZE = 640

for i, row in tqdm(df_test2.iterrows(), total=len(df_test2)):
    image = Image.open(row["image_path"])
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(f"{new_test_path}/{IMAGE_DIR}/{row['id']}.jpg")

100%|██████████| 2281/2281 [00:56<00:00, 40.67it/s]


In [57]:
RESULT_PATH = os.path.abspath("result2")
save_dir = os.path.join(RESULT_PATH, "test2", "exp")

results = model2.predict(
    source='test2/images/', 
    save=True, 
    save_txt=True, 
    project=save_dir,  
    name=""  
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2281 /home/elicer/test2/images/002bc502b8254f939d581c091df31101.jpg: 640x640 3 prominences, 5.9ms
image 2/2281 /home/elicer/test2/images/003f99a30b8442c3ad4ae90eb229bc95.jpg: 640x640 3 prominences, 5.5ms
image 3/2281 /home/elicer/test2/images/00451f2bcf4246269005244d02928a04.jpg: 640x640 2 prominences, 5.4ms
image 4/2281 /home/elicer/test2/images/006958d71a664e24b909f50826dbb4a9.jpg: 640x640 3 prominences, 5.5ms
image 5/2281 /home/elicer/test2/

image 73/2281 /home/elicer/test2/images/094e08703acf418bb1673d5da9e06483.jpg: 640x640 5 prominences, 5.5ms
image 74/2281 /home/elicer/test2/images/0962ddf7bbf04a2888f4edae36a60e8c.jpg: 640x640 3 prominences, 5.4ms
image 75/2281 /home/elicer/test2/images/09db6e8047764aa6a99cf0c40e76657a.jpg: 640x640 3 prominences, 5.4ms
image 76/2281 /home/elicer/test2/images/09e78b9c5baa4a568a8775592efde8a4.jpg: 640x640 2 prominences, 5.5ms
image 77/2281 /home/elicer/test2/images/09f573312a9848fba8436bd51a23f9c4.jpg: 640x640 2 prominences, 5.4ms
image 78/2281 /home/elicer/test2/images/0a4b7dafe98441f98247a9badcff79fa.jpg: 640x640 (no detections), 5.5ms
image 79/2281 /home/elicer/test2/images/0a4cc0cab4234993af83ee2b471dadd2.jpg: 640x640 5 prominences, 5.4ms
image 80/2281 /home/elicer/test2/images/0a53635d68bd4ea7b86ed02b805ffefd.jpg: 640x640 3 prominences, 7.2ms
image 81/2281 /home/elicer/test2/images/0a5f9324b6b54b44b9f8ce5fa0410059.jpg: 640x640 (no detections), 5.5ms
image 82/2281 /home/elicer/test2/

image 150/2281 /home/elicer/test2/images/129811329e264d6f984e5bc1ae4cea4b.jpg: 640x640 3 prominences, 5.4ms
image 151/2281 /home/elicer/test2/images/12a4c99c49b84719909adfc13a7eef9d.jpg: 640x640 3 prominences, 5.4ms
image 152/2281 /home/elicer/test2/images/12bbf9b99a17463b94ceb7328ab8c0bb.jpg: 640x640 1 prominence, 5.7ms
image 153/2281 /home/elicer/test2/images/12d1b4fc0fff4420832804d3e97f449b.jpg: 640x640 2 prominences, 5.3ms
image 154/2281 /home/elicer/test2/images/12e9586c8c0e4f42a2586c6cbe463551.jpg: 640x640 1 prominence, 5.3ms
image 155/2281 /home/elicer/test2/images/12fd04e5e8b54e06bfbef53e17ab124c.jpg: 640x640 1 prominence, 5.3ms
image 156/2281 /home/elicer/test2/images/134b4335ff5d44249e0dcbea2136fb1b.jpg: 640x640 5 prominences, 5.3ms
image 157/2281 /home/elicer/test2/images/134cf01f1b7a4951adec9d5e5f160260.jpg: 640x640 (no detections), 5.3ms
image 158/2281 /home/elicer/test2/images/13741bce452f40028d57cebf392fecd3.jpg: 640x640 2 prominences, 5.4ms
image 159/2281 /home/elicer/t

image 226/2281 /home/elicer/test2/images/1a2cab13d81f4f40aab261f5c04c6463.jpg: 640x640 1 prominence, 5.4ms
image 227/2281 /home/elicer/test2/images/1a7bb83f04ed4f2bbb93b8b6f1861943.jpg: 640x640 1 prominence, 5.3ms
image 228/2281 /home/elicer/test2/images/1a8e5ca28a9e433f8e3888824c52140f.jpg: 640x640 3 prominences, 5.4ms
image 229/2281 /home/elicer/test2/images/1add179cc4a848da88c6808b2cacf6cf.jpg: 640x640 1 prominence, 5.4ms
image 230/2281 /home/elicer/test2/images/1ae4d603639f41ec91c2bd0f73e01c2c.jpg: 640x640 3 prominences, 5.4ms
image 231/2281 /home/elicer/test2/images/1b034e81a3b241f6b83ce13dc20ac340.jpg: 640x640 3 prominences, 5.4ms
image 232/2281 /home/elicer/test2/images/1b45ee8eaa9c43649f138e31bd572090.jpg: 640x640 4 prominences, 7.3ms
image 233/2281 /home/elicer/test2/images/1b67accd3a2b4cf0a1f65ac87bc832b9.jpg: 640x640 1 prominence, 5.5ms
image 234/2281 /home/elicer/test2/images/1b851a81ee1842359acc2a46bb0bb5c8.jpg: 640x640 1 prominence, 5.4ms
image 235/2281 /home/elicer/test2

image 302/2281 /home/elicer/test2/images/23dd34d5e0b74c39aec6130915650818.jpg: 640x640 3 prominences, 6.0ms
image 303/2281 /home/elicer/test2/images/23de32b01f0d4ac5a97e7591160fa15a.jpg: 640x640 1 prominence, 5.5ms
image 304/2281 /home/elicer/test2/images/24004ff3a71a4072bd493c39b8ab8740.jpg: 640x640 (no detections), 5.4ms
image 305/2281 /home/elicer/test2/images/2457e40e040a4e428a9080360b0192ad.jpg: 640x640 2 prominences, 5.4ms
image 306/2281 /home/elicer/test2/images/246655e2ab67480a8321733302d06d01.jpg: 640x640 1 prominence, 5.4ms
image 307/2281 /home/elicer/test2/images/24e6d3c0e0c2435095b9bf19df67f823.jpg: 640x640 2 prominences, 5.3ms
image 308/2281 /home/elicer/test2/images/24ee8571f9644f52af0f58e894a98fb7.jpg: 640x640 5 prominences, 5.4ms
image 309/2281 /home/elicer/test2/images/24f11d09f79d40c8aacd88613908cec5.jpg: 640x640 3 prominences, 5.4ms
image 310/2281 /home/elicer/test2/images/252594294a1845198a41b62fe59241ba.jpg: 640x640 2 prominences, 5.4ms
image 311/2281 /home/elicer/

...

image 1973/2281 /home/elicer/test2/images/d9cb49f8285a4c9385dffc1d7f47567b.jpg: 640x640 4 prominences, 5.4ms
image 1974/2281 /home/elicer/test2/images/da257a9af57c4943b634387810dd3760.jpg: 640x640 3 prominences, 5.4ms
image 1975/2281 /home/elicer/test2/images/da2b6ef0e9f64b2ea67cefc43de1de5a.jpg: 640x640 2 prominences, 5.3ms
image 1976/2281 /home/elicer/test2/images/da31ed8a0f504fc1972bbbb8d7f83e74.jpg: 640x640 1 prominence, 5.4ms
image 1977/2281 /home/elicer/test2/images/da61a982e38c47fe9254b2a0a32ce10d.jpg: 640x640 2 prominences, 5.8ms
image 1978/2281 /home/elicer/test2/images/daf0f0461955457c90416b169ea6814c.jpg: 640x640 3 prominences, 5.4ms
image 1979/2281 /home/elicer/test2/images/daf22e99802a490aa08f41bef7e8b929.jpg: 640x640 6 prominences, 5.6ms
image 1980/2281 /home/elicer/test2/images/db213f414e5c456191a5f3dd8ef850e5.jpg: 640x640 (no detections), 5.4ms
image 1981/2281 /home/elicer/test2/images/dba095ea9b1e4954a89ff617551241f5.jpg: 640x640 (no detections), 5.4ms
image 1982/2281 

image 2049/2281 /home/elicer/test2/images/e54206ddeb2e4c0cb3aad9bd48de9694.jpg: 640x640 3 prominences, 5.4ms
image 2050/2281 /home/elicer/test2/images/e55192c89fb040f2a86cb0fa5b37fe0e.jpg: 640x640 1 prominence, 5.4ms
image 2051/2281 /home/elicer/test2/images/e55cfba55a764c0690abf7fab58eb500.jpg: 640x640 3 prominences, 5.4ms
image 2052/2281 /home/elicer/test2/images/e57b91a86fb8415e907a807c76ac0d96.jpg: 640x640 2 prominences, 5.4ms
image 2053/2281 /home/elicer/test2/images/e5e4f25d7d8c4b7486c31d73061d85cc.jpg: 640x640 2 prominences, 5.4ms
image 2054/2281 /home/elicer/test2/images/e5f56c1f4d6f4089a7d8e1cd2bd67c34.jpg: 640x640 3 prominences, 6.3ms
image 2055/2281 /home/elicer/test2/images/e5ff23f7ce604492a1df7f3aa984f1cb.jpg: 640x640 6 prominences, 5.5ms
image 2056/2281 /home/elicer/test2/images/e6124ca85dc74335a98ec5e519618196.jpg: 640x640 (no detections), 5.4ms
image 2057/2281 /home/elicer/test2/images/e624220e12244149824af6a457f211da.jpg: 640x640 4 prominences, 5.4ms
image 2058/2281 /h

image 2125/2281 /home/elicer/test2/images/eeb8d1463603464e84b988a0a21199ee.jpg: 640x640 (no detections), 5.4ms
image 2126/2281 /home/elicer/test2/images/ef434d1acdf64fafa3f979a52628aab3.jpg: 640x640 2 prominences, 5.4ms
image 2127/2281 /home/elicer/test2/images/ef49897586ba4efa9bdf6bb71644a5e6.jpg: 640x640 1 prominence, 5.4ms
image 2128/2281 /home/elicer/test2/images/ef5794f83494408cb94a6a834f039065.jpg: 640x640 2 prominences, 5.4ms
image 2129/2281 /home/elicer/test2/images/ef65fc12094e4a009ef0d57a2f4226fa.jpg: 640x640 5 prominences, 5.4ms
image 2130/2281 /home/elicer/test2/images/ef70a8a273fe43d5af67b6d0b39ee945.jpg: 640x640 (no detections), 5.4ms
image 2131/2281 /home/elicer/test2/images/ef9554c751534b6eb52e5a594a7f5659.jpg: 640x640 3 prominences, 5.4ms
image 2132/2281 /home/elicer/test2/images/efcc028ef457452fb428c144f6409b4a.jpg: 640x640 2 prominences, 5.4ms
image 2133/2281 /home/elicer/test2/images/efdd41f48f61479d933e2c1f9ebd8d18.jpg: 640x640 1 prominence, 5.4ms
image 2134/2281 /

image 2201/2281 /home/elicer/test2/images/f7009d858b894b9aad6c2c73166cc7e3.jpg: 640x640 3 prominences, 5.5ms
image 2202/2281 /home/elicer/test2/images/f720f71b817a41e68fe46d782835c071.jpg: 640x640 4 prominences, 5.6ms
image 2203/2281 /home/elicer/test2/images/f767105349684393a89ab329c5511f0a.jpg: 640x640 1 prominence, 5.5ms
image 2204/2281 /home/elicer/test2/images/f78d9c7807b64b6c8314c7c050928863.jpg: 640x640 2 prominences, 6.2ms
image 2205/2281 /home/elicer/test2/images/f78f89b40bee4274a1513e333134142b.jpg: 640x640 3 prominences, 5.5ms
image 2206/2281 /home/elicer/test2/images/f7928375cd9b497084d5c41c1725ce76.jpg: 640x640 2 prominences, 5.3ms
image 2207/2281 /home/elicer/test2/images/f7c18be680b744599ddf88d359c79a22.jpg: 640x640 (no detections), 5.3ms
image 2208/2281 /home/elicer/test2/images/f7c4e4932deb4680835551dd31311e67.jpg: 640x640 3 prominences, 5.4ms
image 2209/2281 /home/elicer/test2/images/f7c91ff871a344749d392e3f43d2855a.jpg: 640x640 (no detections), 5.4ms
image 2210/2281 

image 2277/2281 /home/elicer/test2/images/ff9e5e5343bb47db92abee6e0f1b14ab.jpg: 640x640 1 prominence, 5.4ms
image 2278/2281 /home/elicer/test2/images/ffaab2f249be48e58bd3fcecc64524dc.jpg: 640x640 3 prominences, 5.4ms
image 2279/2281 /home/elicer/test2/images/ffaacc2072a444338183c33600455772.jpg: 640x640 4 prominences, 5.4ms
image 2280/2281 /home/elicer/test2/images/ffe58677ad2a47a0bcfa03227973d79f.jpg: 640x640 1 prominence, 5.4ms
image 2281/2281 /home/elicer/test2/images/ffecf60deeac4643a7224122a93072d3.jpg: 640x640 3 prominences, 5.4ms
Speed: 1.6ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/elicer/result2/test2/exp/predict11
1949 labels saved to /home/elicer/result2/test2/exp/predict11/labels


In [58]:
for result in results:
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))
    print('\n{class num:counts} =', uniq_cnt_dict,'\n')
    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model2.names[int(c)])


{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 2} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 2} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_n

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 2} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {} 


{class num:counts} = {} 


{class num:counts} = {2.0: 2} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {} 


{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 4} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 7} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
c

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 2} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 5} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 4} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , class_name = prominence

{class num:counts} = {2.0: 1} 

class num = 2 , class_name = prominence

{class num:counts} = {2.0: 3} 

class num = 2 , class_name = prominence
class num = 2 , class_name = prominence
class num = 2 , 

In [59]:
for i, row in tqdm(df_test2.iterrows(), total=len(df_test2)):
    label_path = os.path.join(RESULT_PATH, "test2", "exp", "predict11", "labels", row["id"] + ".txt")

    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            class_id, c_x, c_y, w, h = map(float, line.split())
            labels.append({"class_id": int(class_id), "x": c_x, "y": c_y, "w": w, "h": h})

    df_test2.at[i, "labels"] = labels
df_test2

100%|██████████| 2281/2281 [00:00<00:00, 6560.86it/s]


,id,image_path,label_path,labels,histograms,cluster
1,e2019257735b4230b8669e8372b9b339,/mnt/elice/dataset/test/images/e2019257735b423...,NaN,"[{'class_id': 2, 'x': 0.155318, 'y': 0.70532, ...",[9.91521478e-01 7.67875165e-02 3.08486763e-02 ...,0
2,2e31937421354646854d599dccc2c80e,/mnt/elice/dataset/test/images/2e3193742135464...,NaN,"[{'class_id': 2, 'x': 0.100141, 'y': 0.435028,...",[9.19836879e-01 3.83912921e-02 1.77258682e-02 ...,0
6,1b86c39d2f4742d6835dfd863df9a0bf,/mnt/elice/dataset/test/images/1b86c39d2f4742d...,NaN,[],[9.89573836e-01 8.96219462e-02 2.48576328e-02 ...,0
11,85d20e1acbc241ef8885345266644703,/mnt/elice/dataset/test/images/85d20e1acbc241e...,NaN,"[{'class_id': 2, 'x': 0.402888, 'y': 0.905335,...",[9.67361867e-01 8.71862173e-02 7.16097504e-02 ...,0
23,76cc809f83494c379a3ae0916ad0ff9b,/mnt/elice/dataset/test/images/76cc809f83494c3...,NaN,"[{'class_id': 2, 'x': 0.0935732, 'y': 0.507972...",[9.90182519e-01 9.02483314e-02 2.36941855e-02 ...,0
...,...,...,...,...,...,...
6358,f95597eb04e94eefa200e4b08ad88659,/mnt/elice/dataset/test/images/f95597eb04e94ee...,NaN,[],[9.89817560e-01 9.13884640e-02 2.52323672e-02 ...,0
6361,a41249df3b054fbfa75cbe827be41a0a,/mnt/elice/dataset/test/images/a41249df3b054fb...,NaN,[],[9.90497351e-01 8.98492709e-02 2.50745341e-02 ...,0
6363,d276c887561b43be82a892ab6ff00378,/mnt/elice/dataset/test/images/d276c887561b43b...,NaN,"[{'class_id': 2, 'x': 0.854838, 'y': 0.671877,...",[9.62262809e-01 6.54817075e-02 5.81028461e-02 ...,0
6364,7a78ae524a5046889713af57538adbee,/mnt/elice/dataset/test/images/7a78ae524a50468...,NaN,"[{'class_id': 2, 'x': 0.224624, 'y': 0.201771,...",[9.23014760e-01 4.35021631e-02 2.36139130e-02 ...,0


## 전체 모으기

In [60]:
submission = []

In [61]:
for i, row in tqdm(df_test0.iterrows(), total=len(df_test0)):
    image_id = row["id"]
    labels = []
    for label in row["labels"]:
        class_id = label["class_id"]
        x = label["x"]
        y = label["y"]
        w = label["w"]
        h = label["h"]

        labels.append({"class_id": class_id, "x": x, "y": y, "w": w, "h": h})
    submission.append({"id": image_id, "labels": labels})

100%|██████████| 2269/2269 [00:00<00:00, 11782.58it/s]


In [62]:
for i, row in tqdm(df_test1.iterrows(), total=len(df_test1)):
    image_id = row["id"]
    labels = []
    for label in row["labels"]:
        class_id = label["class_id"]
        x = label["x"]
        y = label["y"]
        w = label["w"]
        h = label["h"]

        labels.append({"class_id": class_id, "x": x, "y": y, "w": w, "h": h})
    submission.append({"id": image_id, "labels": labels})

100%|██████████| 1819/1819 [00:00<00:00, 11874.47it/s]


In [63]:
for i, row in tqdm(df_test2.iterrows(), total=len(df_test2)):
    image_id = row["id"]
    labels = []
    for label in row["labels"]:
        class_id = label["class_id"]
        x = label["x"]
        y = label["y"]
        w = label["w"]
        h = label["h"]

        labels.append({"class_id": class_id, "x": x, "y": y, "w": w, "h": h})
    submission.append({"id": image_id, "labels": labels})

100%|██████████| 2281/2281 [00:00<00:00, 12689.39it/s]


In [64]:
df_submission = pd.DataFrame(submission)
df_submission

,id,labels
0,4d02b3570c414445b34516800e5eef75,"[{'class_id': 0, 'x': 0.514955, 'y': 0.114202,..."
1,b8c9fd8ff3894f178ee330c4ba285a95,"[{'class_id': 0, 'x': 0.456221, 'y': 0.874741,..."
2,900171cc7641448bb3e546aa68ff16c3,"[{'class_id': 0, 'x': 0.512821, 'y': 0.861852,..."
3,be395257cc8d4fab891d24672a6c1052,"[{'class_id': 0, 'x': 0.439076, 'y': 0.815674,..."
4,1712b54ce3b645058cec2120d60cd1ea,"[{'class_id': 0, 'x': 0.466244, 'y': 0.138028,..."
...,...,...
6364,f95597eb04e94eefa200e4b08ad88659,[]
6365,a41249df3b054fbfa75cbe827be41a0a,[]
6366,d276c887561b43be82a892ab6ff00378,"[{'class_id': 2, 'x': 0.854838, 'y': 0.671877,..."
6367,7a78ae524a5046889713af57538adbee,"[{'class_id': 2, 'x': 0.224624, 'y': 0.201771,..."


In [5]:
df_submission.to_csv("submission.csv", index=False)

In [66]:
df_submission.to_csv("submission_0.859.csv", index=False)

In [3]:
import pandas as pd
df_submission = pd.read_csv('submission_0.859.csv')

In [4]:
df_submission

,id,labels
0,4d02b3570c414445b34516800e5eef75,"[{'class_id': 0, 'x': 0.514955, 'y': 0.114202,..."
1,b8c9fd8ff3894f178ee330c4ba285a95,"[{'class_id': 0, 'x': 0.456221, 'y': 0.874741,..."
2,900171cc7641448bb3e546aa68ff16c3,"[{'class_id': 0, 'x': 0.512821, 'y': 0.861852,..."
3,be395257cc8d4fab891d24672a6c1052,"[{'class_id': 0, 'x': 0.439076, 'y': 0.815674,..."
4,1712b54ce3b645058cec2120d60cd1ea,"[{'class_id': 0, 'x': 0.466244, 'y': 0.138028,..."
...,...,...
6364,f95597eb04e94eefa200e4b08ad88659,[]
6365,a41249df3b054fbfa75cbe827be41a0a,[]
6366,d276c887561b43be82a892ab6ff00378,"[{'class_id': 2, 'x': 0.854838, 'y': 0.671877,..."
6367,7a78ae524a5046889713af57538adbee,"[{'class_id': 2, 'x': 0.224624, 'y': 0.201771,..."
